In [5]:
import pandas as pd
import sys
from __globals import glob
from _utilities import prepare_data, prepare_database
sys.path.append("../tools/")
import _db_tools as db #Personal tool for managing sqlite databases in data science

####Helper Functions####
def limits_gen(df, means, x_tolerance, y_tolerance):
    '''Generate the limit values for a list containing the means in a DataFrame.
    Calculates the total mean for each fiber axis and applies it to the corresponding rows.
    Parameters:
    df (pd.DataFrame): The MEAS dataframe to get its size.
    means (list): A list of means to generate limits for.
    *argv: Variable-length arguments for tolerances. Can be one or two values.
    Returns:
    pd.DataFrame: A DataFrame containing the generated limits.'''
    limits = pd.DataFrame(columns=["LO_LIMIT", "HI_LIMIT"])
    for index in range(int(df.shape[0])):
        if index % 2 == 0:
            low_limit = round(means[0] - x_tolerance, 4)
            high_limit = round(means[0] + x_tolerance, 4)
        else:
            low_limit = round(means[1] - y_tolerance, 4)
            high_limit = round(means[1] + y_tolerance, 4)
        current_limits_df = pd.DataFrame({"LO_LIMIT": [low_limit], "HI_LIMIT": [high_limit]}) #Create a DataFrame with the current low_limit and high_limit values
        limits = pd.concat([limits, current_limits_df], ignore_index=True, axis=0) #Concatenate the current limits DataFrame with the main 'limits' DataFrame
    return limits

def ini_generator_personalized(LIMITS):
    '''Generates a ini file with personalized limits for every mean'''
    class CaseSensitiveConfigParser(configparser.ConfigParser):
        '''A custom class to override optionxform and avoid uppercases being converted to lowercase
        It just works F76 F76 F76 F76 F76'''
        def optionxform(self, optionstr):
            return optionstr
    config = CaseSensitiveConfigParser()
    config.read('../data/template.ini') #Import a template
    keys_list = []
    for section_name in config.sections(): #Get a keys list with the correct uppercased keys
        section = config[section_name]
        keys_list.extend(section.keys())
    HI_LIMIT = LIMITS.iloc[:, 1]
    LO_LIMIT = LIMITS.iloc[:, 0]
    for section in config.sections(): #Iterate through the sections and options in the .ini file
        keys_list = list(config[section].keys())
        j = 0
        for i in range(0, len(keys_list), 2):
            key1 = keys_list[i]
            key2 = keys_list[i + 1]
            col1 = str(LIMITS.iloc[j, 1])
            col2 = str(LIMITS.iloc[j, 0])
            j += 1
            config[section][key1] = col1
            config[section][key2] = col2
    for section in config.sections(): #Print the five first elements of the .ini for a quick check
        print(f"[{section}]")
        i = 0
        for key, value in config.items(section): 
            if i < 5:
                print(f"{key} = {value}")
                i += 1
            else:
                break
        print("...")
    #Save the modified data to a new .ini file
    with open(f'../a2_output/{glob.tooling}.ini', 'w') as configfile:
        for section in config.sections():
            configfile.write(f"[{section}]\n")
            keys = keys_list #Recover the original keys to write them in the .ini file
            for i, key in enumerate(keys):
                configfile.write(f"{key} = {config[section][key]}\n")
                if (i + 1) % 4 == 0 and i < len(keys) - 1: #Insert a blank line every four keys
                    configfile.write("\n")


In [6]:
#Data preparation
dbh = db.SQLite_Data_Extractor("database.db") #Connect to the database
MEAS = dbh.retrieve(glob.tooling) #Get the desired tooling data
dbh.close_conn() 

../database/database.db found.
Table TOP_PASSAT_B9 retrieved succesfully.
Closed connection to: ../database/database.db


In [7]:
#Calculation of the absolute means for fiber x and fiber y
resume = MEAS.transpose().describe() #Transpose the df first due to describe() working in columns.
rough_means = list(resume.iloc[1, :].values)
means = []; means_fbx = []; means_fby = [] #Preallocation
for i, mean in enumerate(rough_means): #Iterates and rounds every mean value
    mean = round(mean, 4)
    means_fbx.append(mean) if i % 2 == 0 else means_fby.append(mean)
    means.append(mean)
abs_mean_fbx = sum(means_fbx) / len(means_fbx)
abs_mean_fby = sum(means_fby) / len(means_fby)
means = [abs_mean_fbx, abs_mean_fby]
print("Means:") 
print("Fiber x: " + str(round(abs_mean_fbx, 4)))
print("Fiber y: " + str(round(abs_mean_fby, 4)))

Means:
Fiber x: 0.3355
Fiber y: 0.3618


In [8]:
LIMITS = limits_gen(MEAS, means, glob.x_tolerance, glob.y_tolerance)
LIMITS

,LO_LIMIT,HI_LIMIT
0,0.3230,0.3480
1,0.3468,0.3768
2,0.3230,0.3480
3,0.3468,0.3768
4,0.3230,0.3480
5,0.3468,0.3768
6,0.3230,0.3480
7,0.3468,0.3768
8,0.3230,0.3480
9,0.3468,0.3768


In [10]:
##Creation of the target dataframe with correct limits
new_column_names = [f'{i+1}' for i in range(len(MEAS.columns))] #Columns renaming
MEAS.columns = new_column_names
new_row_names = []; counter = 1; fiber = "X" #Preallocation
for i in range(0, len(MEAS)): #Rows renaming
    new_row_name = f'Guia_Luz_Blanco_FB{counter}_{fiber}' 
    new_row_names.append(new_row_name)
    if i % 2 == 1: #Alternate the fiber axis and update the fober number every two fibers
        counter += 1
        fiber = "X"
    else:
        fiber = "Y"
MEAS.index = new_row_names
LIMITS.index = new_row_names
target = pd.concat([MEAS, LIMITS], axis=1) #Concatenates the measures and limits
target

,1,2,3,4,5,6,7,8,9,10,...,23,24,25,26,27,28,29,30,LO_LIMIT,HI_LIMIT
Guia_Luz_Blanco_FB1_X,0.3241,0.3242,0.3236,0.3234,0.3246,0.3246,0.3242,0.3243,0.3240,0.3245,...,0.3241,0.3243,0.3240,0.3238,0.3240,0.3235,0.3236,0.3238,0.3230,0.3480
Guia_Luz_Blanco_FB1_Y,0.3532,0.3532,0.3527,0.3526,0.3536,0.3536,0.3534,0.3535,0.3531,0.3533,...,0.3533,0.3532,0.3531,0.3530,0.3529,0.3526,0.3526,0.3529,0.3468,0.3768
Guia_Luz_Blanco_FB2_X,0.3364,0.3362,0.3359,0.3358,0.3364,0.3364,0.3362,0.3363,0.3363,0.3362,...,0.3364,0.3361,0.3362,0.3363,0.3357,0.3358,0.3358,0.3362,0.3230,0.3480
Guia_Luz_Blanco_FB2_Y,0.3610,0.3610,0.3606,0.3606,0.3611,0.3611,0.3609,0.3611,0.3610,0.3609,...,0.3611,0.3608,0.3609,0.3610,0.3605,0.3605,0.3605,0.3609,0.3468,0.3768
Guia_Luz_Blanco_FB3_X,0.3453,0.3458,0.3453,0.3454,0.3456,0.3457,0.3457,0.3457,0.3456,0.3454,...,0.3454,0.3453,0.3455,0.3453,0.3449,0.3455,0.3456,0.3451,0.3230,0.3480
Guia_Luz_Blanco_FB3_Y,0.3704,0.3707,0.3703,0.3704,0.3706,0.3706,0.3706,0.3707,0.3705,0.3704,...,0.3705,0.3703,0.3704,0.3704,0.3701,0.3703,0.3704,0.3702,0.3468,0.3768
Guia_Luz_Blanco_FB4_X,0.3220,0.3219,0.3218,0.3216,0.3216,0.3216,0.3217,0.3214,0.3214,0.3213,...,0.3216,0.3214,0.3214,0.3212,0.3211,0.3210,0.3209,0.3209,0.3230,0.3480
Guia_Luz_Blanco_FB4_Y,0.3501,0.3500,0.3498,0.3498,0.3497,0.3497,0.3497,0.3496,0.3495,0.3494,...,0.3497,0.3494,0.3494,0.3493,0.3492,0.3491,0.3491,0.3490,0.3468,0.3768
Guia_Luz_Blanco_FB5_X,0.3346,0.3347,0.3347,0.3346,0.3348,0.3345,0.3347,0.3342,0.3342,0.3343,...,0.3343,0.3347,0.3344,0.3340,0.3342,0.3343,0.3341,0.3343,0.3230,0.3480
Guia_Luz_Blanco_FB5_Y,0.3605,0.3605,0.3605,0.3604,0.3606,0.3603,0.3605,0.3601,0.3601,0.3601,...,0.3602,0.3604,0.3602,0.3599,0.3600,0.3601,0.3599,0.3600,0.3468,0.3768


In [11]:
##Data export
confirmation = input("Do you want to export the data to a new Target.xlsx file? (y/n): ").strip().lower()
if confirmation == 'y':
    target.to_excel('../a2_output/Target_with_limits.xlsx', index=True, header=True)
    #os.startfile('../a2_output/Target_with_limits.xlsx')
    print('Data exported in: ../a2_output/Target_with_limits.xlsx')
else:
    print("Operation canceled.")

Data exported in: ../a2_output/Target_with_limits.xlsx


In [12]:
##Update of the database table
confirmation = input("Do you want to export the new limits to your database (y/n): ").strip().lower()
if confirmation == 'y':
    prepare_database(LIMITS, glob.tooling+"_limits") #Store a df inside the database of the project
else:
    print("Operation canceled.")

../database/database.db found.
Dataframe stored as *TOP_PASSAT_B9_limits*
../database/database.db actual contents:
    TOP_PASSAT_B9
    TOP_PASSAT_B9_limits
